In [ ]:
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import requests

**画像の著作権がわからないので実行結果を消して commit する。**

In [ ]:
# https://www.chiibakun.jp/?pid=65459171
DIRPATH_DATA = Path("../data/")
filepath_img = DIRPATH_DATA / "65459171.png"
filepath_img

In [ ]:
image = cv2.imread(filepath_img)
image.shape

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# 二値化
threshold = 200
ret, binary = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)

In [ ]:
# 輪郭検出
contours, hierarchy = cv2.findContours(
    binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
)

In [ ]:
# 輪郭だけを描画（黒色で描画）
image_contours = cv2.cvtColor(
    cv2.drawContours(
        np.full(image.shape, 255, dtype=np.uint8), contours, -1, (0, 0, 0), 3
    ),
    cv2.COLOR_BGR2GRAY,
)

In [ ]:
# 描画
plt.imshow(image_contours)

In [ ]:
num_labels, label_image, stats, centroids = cv2.connectedComponentsWithStats(
    image_contours
)

In [ ]:
stats  # オブジェクトのバウンディングボックス（開始点の x 座標、y 座標、幅、高さ）とオブジェクトのサイズ。

In [ ]:
idx_label_chi_ba_kun = sorted(
    range(num_labels), key=lambda idx: stats[idx][-1]
)[::-1][2]
image_chi_ba_kun_rough = label_image == idx_label_chi_ba_kun
plt.imshow(image_chi_ba_kun_rough)

In [ ]:
# 微調整 (うちわ分)
image_chi_ba_kun_rough[355:, 152:] = 0
plt.imshow(image_chi_ba_kun_rough)

In [ ]:
# 輪郭検出
contours, hierarchy = cv2.findContours(
    image_chi_ba_kun_rough.astype(np.uint8),
    cv2.RETR_TREE,
    cv2.CHAIN_APPROX_SIMPLE,
)

# 輪郭だけを描画（黒色で描画）
image_contours_2 = cv2.cvtColor(
    cv2.drawContours(
        np.full(image.shape, 255, dtype=np.uint8), contours, -1, (0, 0, 0), 3
    ),
    cv2.COLOR_BGR2GRAY,
)

In [ ]:
num_labels, label_image, stats, centroids = cv2.connectedComponentsWithStats(
    image_contours_2
)

In [ ]:
idx_label_chi_ba_kun_only = max(
    range(num_labels), key=lambda idx: stats[idx][-1]
)
image_chi_ba_kun = label_image != idx_label_chi_ba_kun_only
plt.imshow(image_chi_ba_kun)

In [ ]:
fig, ax = plt.subplots()
mappable = ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
ax.scatter(*centroids[idx_label_chi_ba_kun_only], marker="*", color="black")
ax.annotate(
    "$G$",
    xy=centroids[idx_label_chi_ba_kun_only],
    xytext=tuple(map(lambda x: x + 1, centroids[idx_label_chi_ba_kun_only])),
)
fig.tight_layout()

In [ ]:
fig, axes = plt.subplots(
    ncols=2, dpi=300, figsize=(6.4, 3.2), sharex=True, sharey=True
)
axes[0].imshow(image_chi_ba_kun)
axes[1].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
for ax in axes:
    ax.scatter(
        *centroids[idx_label_chi_ba_kun_only], marker="*", color="black"
    )
    ax.annotate(
        "$G$",
        xy=centroids[idx_label_chi_ba_kun_only],
        xytext=tuple(
            map(lambda x: x + 5, centroids[idx_label_chi_ba_kun_only])
        ),
        ha="left",
        va="top",
    )
fig.tight_layout()